<a href="https://colab.research.google.com/github/AbishekArunachalam/Image-Captioning--Deep-Learning-model-/blob/master/DeepLearningAT3_Building_an_Image_Captioning_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>